In [27]:
import requests
from bs4 import BeautifulSoup
import os

In [28]:
ICIRR_URL = 'https://www.icirr.org/'

In [33]:
def scrape_urls(url, depth):
    # Send a GET request to the URL
    response = requests.get(url)
    # Check if request was unsuccessful
    if response.status_code != 200:
        print("Failed to retrieve page. Status code:", response.status_code)
        return set()

    # Check if depth is 0, return an empty set
    if depth == 0:
        return set()

    result = set()
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find and extract information you need from the page
    # For example, find all links on the page
    links = soup.find_all('a')
    for link in links:
        # Get the URL of the link
        href = str(link.get('href'))
        if not href or "icirr.org" not in href or "ICIRR-Staff" in href:
            continue
        result.add(href)
        # If the link is within the same domain, crawl it recursively
        if depth > 1:
            result.update(scrape_urls(href, depth - 1))
    return result

def save_urls(urls):
    # Save the URLs to a file
    with open(os.path.join('parsed_html', 'urls.txt'), 'w', encoding='utf-8') as f:
        for url in urls:
            f.write(url + '\n')
    
def save_page(url):
    # Send a GET request to the URL
    response = requests.get(url)
    # Check if request was unsuccessful
    if response.status_code != 200:
        print("Failed to retrieve page. Status code:", response.status_code)
        return

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Get the title of the webpage
    title = hash(url)
    
    # Create a directory to store pages if it doesn't exist
    if not os.path.exists('web_scraped'):
        os.makedirs('web_scraped')

    # Save the page to a file
    with open(os.path.join('web_scraped', f"{title}.html"), 'w', encoding='utf-8') as f:
        f.write(response.text)

def load_urls():
    urls = set()
    # Load the URLs from the file
    with open(os.path.join('web_scraped', 'urls.txt'), 'r', encoding='utf-8') as f:
        for line in f:
            urls.add(line.strip())
    return urls

In [30]:
scraped_urls = scrape_urls(ICIRR_URL, 2)
save_urls(scraped_urls)

FileNotFoundError: [Errno 2] No such file or directory: 'web_scraped/urls.txt'

In [34]:
save_urls(scraped_urls)